In [143]:
import os
import sys
import numpy as np
import bigbadbrain as bbb
import pandas as pd
from lxml import etree, objectify
from xml.etree import ElementTree as ET

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 100)

## One-time creation of empty master experiment dataframe

In [184]:
columns = ['fly_id',
           'expt_id',
           'date',
           'time',
           'scan_type',
           'genotype',
           'brain_area',
           'visual_stimuli',
           'fictrac',
           'circ_on',
           'circ_off',
           'gender',
           'age',
           'expt_notes',
           'temp',
           'fly_notes',
           'laser_power',
           'PMT_green',
           'PMT_red',
           'x_voxel_size',
           'y_voxel_size',
           'z_voxel_size']

master_expt = pd.DataFrame(columns=columns)
master_expt_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/master_expt.pkl'

master_expt.to_pickle(master_expt_path)

## Development of expt adding code

In [185]:
# From fly_builder.py

def get_datetime_from_xml(xml_file):
    print('Getting datetime from {}'.format(xml_file))
    tree = ET.parse(xml_file)
    root = tree.getroot()
    datetime = root.get('date')
    # will look like "4/2/2019 4:16:03 PM" to start

    # Get dates
    date = datetime.split(' ')[0]
    month = date.split('/')[0]
    day = date.split('/')[1]
    year = date.split('/')[2]

    # Get times
    time = datetime.split(' ')[1]
    hour = time.split(':')[0]
    minute = time.split(':')[1]
    second = time.split(':')[2]

    # Convert from 12 to 24 hour time
    am_pm = datetime.split(' ')[-1]
    if am_pm == 'AM' and hour == '12':
        hour = str(00)
    elif am_pm == 'AM':
        pass
    elif am_pm == 'PM' and hour == '12':
        pass
    else:
        hour = str(int(hour) + 12)

    # Add zeros if needed
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    if len(hour) == 1:
        hour = '0' + hour

    # Combine
    datetime_str = year + month + day + '-' + hour + minute + second
    datetime_int = int(year + month + day + hour + minute + second)
    datetime_dict = {'year': year,
                     'month': month,
                     'day': day,
                     'hour': hour,
                     'minute': minute,
                     'second': second}

    return datetime_str, datetime_int, datetime_dict

In [186]:
def add_fly_to_metadata(fly_folder):
    # For each fly, need to handle multiple experiments
    
    # Load fly metadata
    xml_file = os.path.join(fly_folder, 'fly.xml')
    fly = load_xml(xml_file)
    
    # Need to add each experiment
    # First get correct expt order based on expt time
    expts = []
    for item in os.listdir(fly_folder):
        if os.path.isdir(os.path.join(fly_folder, item)):
            if 'anatomy' in item or 'functional' in item:
                expts.append(item)
    
    expts_sorted, datetimes_sorted = order_expts(fly_folder, expts)
    
    for i in range(len(expts)):
        expt_folder = os.path.join(fly_folder, expts_sorted[i])
        add_expt_to_metadata(fly, expt_folder, datetimes_sorted[i], i)
                
def order_expts(fly_folder, expts):
    # For each expt, get it's time
    datetimes = []
    print('expts pre-sort: {}'.format(expts))
    for expt in expts:
        if 'anatomy' in expt:
            xml_file = os.path.join(fly_folder, expt, 'anatomy.xml')
            datetime_str,_,_ = get_datetime_from_xml(xml_file)
        if 'functional' in expt:
            xml_file = os.path.join(fly_folder, expt, 'imaging', 'functional.xml')
            datetime_str,_,_ = get_datetime_from_xml(xml_file)
        datetimes.append(datetime_str)
    
    expts_sorted = [x for _,x in sorted(zip(datetimes,expts))]
    print('expts_sorted: {}'.format(expts_sorted))
    datetimes_sorted = sorted(datetimes)
    print('datetimes_sorted: {}'.format(datetimes_sorted))
    return expts_sorted, datetimes_sorted

def add_expt_to_metadata(fly, expt_folder, datetime, expt_id):    
    # Load expt metadata
    xml_file = os.path.join(expt_folder, 'expt.xml')
    expt = load_xml(xml_file)
    
    # Load visual metadata
    try:
        xml_file = os.path.join(expt_folder, 'visual', 'visual.xml')
        visual = load_xml(xml_file)
    except:
        visual = None
        print('Visual metadata not found.')
    
    # Load fictrac metadata
    try:
        xml_file = os.path.join(expt_folder, 'fictrac', 'fictrac.xml')
        fictrac = load_xml(xml_file)
        
        # UPDATE THIS WHEN FICTRAC META CONTAINS REAL INFO
        fictrac = True
    except:
        fictrac = None
        print('Fictrac metadata not found.')
    
    # Load scan metadata
    # First try to load from imaging folder (in the case of functional)
    try:
        xml_file = os.path.join(expt_folder, 'imaging', 'scan.xml')
        scan = load_xml(xml_file)
    # Next try to load from expt folder (in the case of anatomy)
    except:
        xml_file = os.path.join(expt_folder, 'scan.xml')
        scan = load_xml(xml_file)
    
    # Load experiment master dataframe
    master_expt_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/master_expt.pkl'
    master_expt = pd.read_pickle(master_expt_path)
    
    # Get scan type
    if 'functional' in expt_folder:
        scan_type = 'functional'
    if 'anatomy' in expt_folder:
        scan_type = 'anatomy'
    
    # Get datetime
    date = datetime.split('-')[0]
    time = datetime.split('-')[1]
    
    # Get fly_id
    fly_folder = os.path.split(os.path.split(expt_folder)[0])[-1]
    fly_id = fly_folder.split('_')[-1]

    # Append the new row
    new_row = {'fly_id': fly_id,
               'expt_id': expt_id,
               'date': date,
               'time': time,
               'circ_on': str(fly.circadian_on),
               'circ_off': str(fly.circadian_off),
               'gender': str(fly.gender),
               'age': str(fly.age),
               'genotype': str(fly.genotype),
               'brain_area': str(expt.brain_area),
               'expt_notes': str(expt.notes),
               'scan_type': str(scan_type),
               'temp': str(fly.temp),
               'fly_notes': str(fly.notes),
               'laser_power': str(scan.power),
               'PMT_green': str(scan.PMT_green),
               'PMT_red': str(scan.PMT_red),
               'x_voxel_size': str(scan.x_voxel_size),
               'y_voxel_size': str(scan.y_voxel_size),
               'z_voxel_size': str(scan.z_voxel_size)}
    
    if visual is not None:
        new_row['visual_stimuli'] = str(visual.unique_stimuli)
    if fictrac is not None:
        new_row['fictrac'] = fictrac
               
    master_expt = master_expt.append(new_row, ignore_index=True)
    
    # Save
    master_expt.to_pickle(master_expt_path)

def load_xml(file):
    tree = objectify.parse(file)
    root = tree.getroot()
    return root

In [187]:
fly_folder = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_20190506-095420_50'
add_fly_to_metadata(fly_folder)

expts pre-sort: ['optic_functional_2', 'optic_anatomy', 'optic_functional_1']
Getting datetime from /oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_20190506-095420_50/optic_functional_2/imaging/functional.xml
Getting datetime from /oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_20190506-095420_50/optic_anatomy/anatomy.xml
Getting datetime from /oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/fly_20190506-095420_50/optic_functional_1/imaging/functional.xml
expts_sorted: ['optic_functional_1', 'optic_functional_2', 'optic_anatomy']
datetimes_sorted: ['20190506-095420', '20190506-105931', '20190506-121216']
Visual metadata not found.
Fictrac metadata not found.


In [190]:
master_expt_path = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset/master_expt.pkl'
master_expt = pd.read_pickle(master_expt_path)

In [191]:
master_expt

,fly_id,expt_id,date,time,scan_type,genotype,brain_area,visual_stimuli,fictrac,circ_on,circ_off,gender,age,expt_notes,temp,fly_notes,laser_power,PMT_green,PMT_red,x_voxel_size,y_voxel_size,z_voxel_size
0,50,0,20190506,095420,functional,GCaMP6f,optic,"[{'name': 'ConstantBackground', 'background': 0}, {'name': 'ConstantBackground', 'background': 1}]",True,23,11,female,6,None,25,None,200,700,600,1.63198312052867,1.63198312052867,5.0
1,50,1,20190506,105931,functional,GCaMP6f,optic-zoom,"[{'name': 'ConstantBackground', 'background': 0}, {'name': 'ConstantBackground', 'background': 1}]",True,23,11,female,6,fictrac failed twice...,25,None,200,700,600,1.04446919713835,1.04446919713835,5.0
2,50,2,20190506,121216,anatomy,GCaMP6f,optic,NaN,NaN,23,11,female,6,None,25,None,300,700,620,1.00429730494072,1.00429730494072,1
3,33,0,20190423,112959,functional,99/1_GCaMP6f,optic,"[{'name': 'SineGrating', 'angle': 180, 'period': 40, 'rate': 100, 'color': 1}, {'name': 'SineGrating', 'angle': 270, 'period': 40, 'rate': 100, 'color': 1}, {'name': 'SineGrating', 'angle': 0, 'period': 40, 'rate': 100, 'color': 1}, {'name': 'SineGrating', 'angle': 90, 'period': 40, 'rate': 100, 'color': 1}]",True,1,13,female,7,None,25,None,300,750,550,1.00429730494072,1.00429730494072,5.0
4,33,1,20190423,121053,functional,99/1_GCaMP6f,central,"[{'name': 'SineGrating', 'angle': 180, 'period': 40, 'rate': 100, 'color': 1}, {'name': 'SineGrating', 'angle': 270, 'period': 40, 'rate': 100, 'color': 1}, {'name': 'SineGrating', 'angle': 0, 'period': 40, 'rate': 100, 'color': 1}, {'name': 'SineGrating', 'angle': 90, 'period': 40, 'rate': 100, 'color': 1}]",True,1,13,female,7,None,25,None,300,750,550,1.00429730494072,1.00429730494072,5.0
